# Advanced Infrastructure

## LLM Servers, Open-Source Models, and Agent-to-Agent Communication

This notebook explores the infrastructure layer of AI engineering — self-hosted LLM servers, open-source model deployment, and patterns for agents to communicate with each other. We cover the full stack from local development with Ollama to production deployment with Together AI, plus exposing your agents as MCP servers for other agents to consume.

```
┌─────────────────────────────────────────────────────────────┐
│                  Advanced Infrastructure                    │
│                                                             │
│   ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐    │
│   │  Ollama  │  │  vLLM /  │  │ Together │  │   MCP    │    │
│   │  Local   │  │   TGI    │  │    AI    │  │  Server  │    │
│   │   Dev    │  │Production│  │  Hosting │  │ Exposure │    │
│   └────┬─────┘  └────┬─────┘  └────┬─────┘  └────┬─────┘    │
│        │             │             │             │          │
│        v             v             v             v          │
│   Fast local     High-throughput  Managed       Agent-to-   │
│   iteration      inference        open models   agent APIs  │
└─────────────────────────────────────────────────────────────┘
```

Topics covered:
- LLM server fundamentals (model serving, inference optimization, cost analysis)
- Popular LLM servers (vLLM, TGI, Ollama)
- Open-source LLMs and embedding models (Llama 4, Mixtral, Qwen3, DeepSeek)
- Deploying open models with Together AI
- Agent-to-agent communication patterns (request-response, pub-sub, event sourcing)
- Exposing agents via MCP (Model Context Protocol)

## Setup and Imports

Make sure you have the required packages installed:

```bash
uv sync
```

This notebook uses optional dependencies:
- **Ollama** — For local model inference (install from ollama.com)
- **Together AI** — For hosted open-source models (set `TOGETHER_API_KEY`)

In [ ]:
import os
import sys
import json
import time
import uuid
import urllib.request
from datetime import datetime
from typing import Optional

from dotenv import load_dotenv
from pydantic import BaseModel
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Ollama integration (optional - for local development)
try:
    from langchain_ollama import ChatOllama
    OLLAMA_AVAILABLE = True
except ImportError:
    OLLAMA_AVAILABLE = False
    print("langchain_ollama not installed. Ollama features will be skipped.")

# Together AI integration (optional - for open-source model hosting)
try:
    from together import Together
    TOGETHER_AVAILABLE = True
except ImportError:
    TOGETHER_AVAILABLE = False
    print("together SDK not installed. Together AI features will be skipped.")

# MCP imports
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Load environment variables
load_dotenv()

# Check for API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables")


def check_ollama_running() -> bool:
    """Check if Ollama server is running locally."""
    try:
        urllib.request.urlopen("http://localhost:11434/api/tags", timeout=2)
        return True
    except Exception:
        return False


OLLAMA_RUNNING = check_ollama_running() if OLLAMA_AVAILABLE else False

# Initialize default LLM (OpenAI as fallback)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("Setup complete!")
print(f"  OpenAI: Ready")
print(f"  Ollama: {'Running' if OLLAMA_RUNNING else 'Not available'}")
print(f"  Together AI: {'Ready' if TOGETHER_API_KEY else 'Missing API key'}")

## LLM Servers - Model Serving Fundamentals

An LLM server is the infrastructure layer that hosts and serves language model inference. Understanding the fundamentals helps you make informed decisions about self-hosting vs. using managed APIs.

### Key Concepts

**Inference Optimization Techniques:**
- **Quantization** — Reduce model precision (FP32 → FP16 → INT8 → INT4) to decrease memory usage and increase throughput
- **Continuous Batching** — Process multiple requests together, dynamically adding/removing requests as they complete
- **KV Cache** — Store computed key-value pairs from attention layers to avoid redundant computation
- **PagedAttention** — Manage KV cache memory like virtual memory pages for efficient utilization

In [ ]:
class InferenceConfig(BaseModel):
    """Configuration for LLM inference optimization."""
    max_batch_size: int = 32
    max_sequence_length: int = 4096
    kv_cache_dtype: str = "auto"  # "auto", "fp16", "fp8"
    quantization: Optional[str] = None  # "awq", "gptq", "fp8"
    tensor_parallel_size: int = 1
    gpu_memory_utilization: float = 0.9


# Example configurations for different scenarios
CONFIGS = {
    "development": InferenceConfig(
        max_batch_size=4,
        max_sequence_length=2048,
        quantization="awq",
        tensor_parallel_size=1,
    ),
    "production_single_gpu": InferenceConfig(
        max_batch_size=32,
        max_sequence_length=8192,
        quantization="fp8",
        tensor_parallel_size=1,
        gpu_memory_utilization=0.95,
    ),
    "production_multi_gpu": InferenceConfig(
        max_batch_size=64,
        max_sequence_length=32768,
        quantization=None,  # Full precision for quality
        tensor_parallel_size=4,
    ),
}

print("Inference Configurations:")
for name, config in CONFIGS.items():
    print(f"\n{name}:")
    print(f"  Batch size: {config.max_batch_size}")
    print(f"  Max sequence: {config.max_sequence_length}")
    print(f"  Quantization: {config.quantization or 'None (full precision)'}")
    print(f"  Tensor parallel: {config.tensor_parallel_size} GPU(s)")

### Cost Analysis: Self-Hosting vs. APIs

The decision to self-host depends on your volume, latency requirements, and operational capacity.

In [ ]:
class CostComparison(BaseModel):
    """Compare self-hosting vs API costs."""
    provider: str
    model: str
    input_cost_per_1m: float  # Cost per 1M input tokens
    output_cost_per_1m: float  # Cost per 1M output tokens
    monthly_fixed_cost: float = 0.0  # GPU rental, etc.
    
    def monthly_cost(self, input_tokens: int, output_tokens: int) -> float:
        """Calculate monthly cost for given token volumes."""
        input_cost = (input_tokens / 1_000_000) * self.input_cost_per_1m
        output_cost = (output_tokens / 1_000_000) * self.output_cost_per_1m
        return input_cost + output_cost + self.monthly_fixed_cost


# Cost comparison data (February 2026 pricing)
COST_OPTIONS = [
    CostComparison(
        provider="OpenAI",
        model="gpt-4o-mini",
        input_cost_per_1m=0.15,
        output_cost_per_1m=0.60,
    ),
    CostComparison(
        provider="Together AI",
        model="Llama-4-Scout-17B",
        input_cost_per_1m=0.10,
        output_cost_per_1m=0.20,
    ),
    CostComparison(
        provider="Self-hosted (A100 80GB)",
        model="Llama-4-Scout-17B",
        input_cost_per_1m=0.0,
        output_cost_per_1m=0.0,
        monthly_fixed_cost=2000.0,  # ~$2.78/hr * 720 hrs
    ),
]


def analyze_costs(monthly_input_tokens: int, monthly_output_tokens: int):
    """Analyze costs across providers for given token volumes."""
    print(f"Cost Analysis for {monthly_input_tokens:,} input + {monthly_output_tokens:,} output tokens/month")
    print("-" * 60)
    
    results = []
    for option in COST_OPTIONS:
        cost = option.monthly_cost(monthly_input_tokens, monthly_output_tokens)
        results.append((option.provider, option.model, cost))
        print(f"{option.provider} ({option.model}): ${cost:,.2f}/month")
    
    cheapest = min(results, key=lambda x: x[2])
    print(f"\nBest option: {cheapest[0]} at ${cheapest[2]:,.2f}/month")


# Compare at different scales
print("=== Low Volume (startup/development) ===")
analyze_costs(1_000_000, 500_000)  # 1M input, 500K output

print("\n=== Medium Volume (production app) ===")
analyze_costs(50_000_000, 25_000_000)  # 50M input, 25M output

print("\n=== High Volume (enterprise scale) ===")
analyze_costs(500_000_000, 250_000_000)  # 500M input, 250M output

## Popular LLM Servers

Three main options dominate the LLM serving landscape:

| Server | Best For | Key Features |
|--------|----------|-------------|
| **vLLM** | High-throughput production | PagedAttention, continuous batching, OpenAI-compatible API |
| **TGI** | Hugging Face ecosystem | Multi-backend support, quantization, Rust performance |
| **Ollama** | Local development | Simple CLI, easy model management, cross-platform |

### vLLM for High-Throughput Serving

vLLM (v0.16.0 as of February 2026) is the industry standard for production LLM serving. It uses PagedAttention to achieve 2-4x higher throughput than naive implementations.

**Starting a vLLM server:**
```bash
# Install vLLM
pip install vllm

# Start server with OpenAI-compatible API
vllm serve meta-llama/Llama-4-Scout-17B-16E-Instruct \
    --tensor-parallel-size 2 \
    --max-model-len 8192 \
    --quantization awq \
    --api-key token-abc123
```

**Key vLLM 0.16 features:**
- RTX Blackwell (SM120) support
- PyTorch 2.10 compatibility
- EAGLE3 speculative decoding
- FP8 MoE kernel support

In [ ]:
# vLLM configuration example (reference only - requires GPU)
VLLM_CONFIG = {
    "model": "meta-llama/Llama-4-Scout-17B-16E-Instruct",
    "tensor_parallel_size": 2,
    "max_model_len": 8192,
    "quantization": "awq",
    "gpu_memory_utilization": 0.9,
    "enable_chunked_prefill": True,
    "max_num_seqs": 256,
}

print("vLLM Configuration (for production deployment):")
print(json.dumps(VLLM_CONFIG, indent=2))

print("\n" + "="*50)
print("To start vLLM server:")
print("="*50)
print(f"vllm serve {VLLM_CONFIG['model']} \\")
print(f"    --tensor-parallel-size {VLLM_CONFIG['tensor_parallel_size']} \\")
print(f"    --max-model-len {VLLM_CONFIG['max_model_len']} \\")
print(f"    --quantization {VLLM_CONFIG['quantization']} \\")
print(f"    --api-key your-api-key")

### Text Generation Inference (TGI)

TGI (v3.x) from Hugging Face is a production-grade inference server built in Rust and Python.

**Starting TGI with Docker:**
```bash
docker run --gpus all --shm-size 1g -p 8080:80 \
    -v /data:/data \
    ghcr.io/huggingface/text-generation-inference:3.0 \
    --model-id meta-llama/Llama-4-Scout-17B-16E-Instruct \
    --quantize awq
```

**Key TGI v3 features:**
- Multi-backend support (vLLM, TensorRT-LLM)
- Zero-config mode for easy deployment
- Torch 2.7 and CUDA 12.8 support

In [ ]:
# TGI Docker command reference
TGI_DOCKER_CMD = """
docker run --gpus all --shm-size 1g -p 8080:80 \\
    -v /data:/data \\
    -e HF_TOKEN=$HF_TOKEN \\
    ghcr.io/huggingface/text-generation-inference:3.0 \\
    --model-id meta-llama/Llama-4-Scout-17B-16E-Instruct \\
    --quantize awq \\
    --max-concurrent-requests 128 \\
    --max-input-tokens 4096 \\
    --max-total-tokens 8192
"""

print("TGI Docker Deployment:")
print(TGI_DOCKER_CMD)

print("\nTGI vs vLLM Comparison:")
print("-" * 50)
print("vLLM: Better raw throughput, more quantization options")
print("TGI: Easier Hugging Face integration, multi-backend support")

### Ollama for Local Development

Ollama (v0.16.1) makes local LLM development simple. Install from [ollama.com](https://ollama.com) and run models with a single command.

**Getting started:**
```bash
# Start Ollama server
ollama serve

# Pull a model
ollama pull llama3.2

# Run interactively
ollama run llama3.2
```

In [ ]:
def test_ollama_local():
    """Test Ollama with a local model."""
    if not OLLAMA_RUNNING:
        print("Ollama not running.")
        print("To use Ollama:")
        print("  1. Install from https://ollama.com")
        print("  2. Run: ollama serve")
        print("  3. Pull a model: ollama pull llama3.2")
        return None
    
    print("Testing Ollama with llama3.2...")
    ollama_llm = ChatOllama(
        model="llama3.2",
        temperature=0,
        base_url="http://localhost:11434"
    )
    
    response = ollama_llm.invoke("What is the capital of France? Answer in one word.")
    print(f"Ollama response: {response.content}")
    return response


# Test Ollama if available
test_ollama_local()

### OpenAI-Compatible APIs

All three servers (vLLM, TGI, Ollama) expose OpenAI-compatible APIs. This means you can use the same client code to connect to any of them.

In [ ]:
def create_openai_compatible_client(base_url: str, api_key: str = "not-needed") -> OpenAI:
    """Create a client for any OpenAI-compatible server."""
    return OpenAI(base_url=base_url, api_key=api_key)


def test_openai_compatible(base_url: str, model: str, api_key: str = "not-needed"):
    """Test an OpenAI-compatible endpoint."""
    client = create_openai_compatible_client(base_url, api_key)
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": "Say hello in exactly 3 words."}],
        max_tokens=20,
    )
    return response.choices[0].message.content


# Example: Connecting to Ollama via OpenAI SDK
if OLLAMA_RUNNING:
    print("Testing Ollama via OpenAI-compatible API...")
    result = test_openai_compatible(
        base_url="http://localhost:11434/v1",
        model="llama3.2"
    )
    print(f"Response: {result}")
else:
    print("Ollama not running. Example configuration:")
    print("  base_url = 'http://localhost:11434/v1'")
    print("  model = 'llama3.2'")

print("\nOther OpenAI-compatible endpoints:")
print("  vLLM: http://localhost:8000/v1")
print("  TGI:  http://localhost:8080/v1")

## Using Open-Source LLMs

The open-source LLM landscape has matured significantly. As of February 2026, several models compete with proprietary options on quality while offering significant cost savings.

In [ ]:
class ModelInfo(BaseModel):
    """Information about an open-source model."""
    name: str
    provider: str
    parameters: str
    context_length: int
    strengths: list[str]
    best_for: list[str]
    license: str


# Latest open-source models (February 2026)
OPEN_SOURCE_MODELS = [
    ModelInfo(
        name="Llama-4-Scout-17B-16E-Instruct",
        provider="Meta",
        parameters="17B active / 109B total (MoE)",
        context_length=10_000_000,  # 10M tokens!
        strengths=["Massive context window", "Efficient MoE", "Multimodal"],
        best_for=["Long documents", "Codebase analysis", "Research"],
        license="Llama 4 Community"
    ),
    ModelInfo(
        name="Llama-4-Maverick-17B-128E-Instruct",
        provider="Meta",
        parameters="17B active / 400B total (MoE)",
        context_length=128_000,
        strengths=["Highest quality", "Complex reasoning", "Multimodal"],
        best_for=["Research", "Complex analysis", "Creative tasks"],
        license="Llama 4 Community"
    ),
    ModelInfo(
        name="Mixtral-8x22B-Instruct-v0.1",
        provider="Mistral AI",
        parameters="39B active / 141B total (MoE)",
        context_length=64_000,
        strengths=["Multilingual", "Cost-effective", "Apache 2.0 license"],
        best_for=["European languages", "Translation", "General assistant"],
        license="Apache 2.0"
    ),
    ModelInfo(
        name="Qwen3-72B-Instruct",
        provider="Alibaba",
        parameters="72B dense",
        context_length=128_000,
        strengths=["Asian languages", "Math", "Coding"],
        best_for=["Multilingual apps", "Technical tasks", "STEM"],
        license="Qwen License"
    ),
    ModelInfo(
        name="DeepSeek-V3.2",
        provider="DeepSeek",
        parameters="671B total (MoE)",
        context_length=128_000,
        strengths=["Coding", "Math", "Reasoning"],
        best_for=["Developer tools", "Technical Q&A", "Code generation"],
        license="DeepSeek License"
    ),
]


print("Open-Source Model Comparison (February 2026)")
print("=" * 70)
for model in OPEN_SOURCE_MODELS:
    print(f"\n{model.name} ({model.provider})")
    print(f"  Parameters: {model.parameters}")
    print(f"  Context: {model.context_length:,} tokens")
    print(f"  License: {model.license}")
    print(f"  Strengths: {', '.join(model.strengths)}")
    print(f"  Best for: {', '.join(model.best_for)}")

In [ ]:
def select_model_for_task(
    task_type: str,
    context_needed: int = 4096,
    needs_multilingual: bool = False,
    prefer_open_license: bool = False
) -> ModelInfo:
    """Recommend a model based on task requirements."""
    
    candidates = OPEN_SOURCE_MODELS.copy()
    
    # Filter by context length
    candidates = [m for m in candidates if m.context_length >= context_needed]
    
    # Filter by license if needed
    if prefer_open_license:
        candidates = [m for m in candidates if "Apache" in m.license]
    
    # Score by task fit
    def score(model: ModelInfo) -> int:
        s = 0
        task_lower = task_type.lower()
        for use in model.best_for:
            if task_lower in use.lower() or use.lower() in task_lower:
                s += 2
        if needs_multilingual and "multilingual" in " ".join(model.strengths).lower():
            s += 1
        return s
    
    if not candidates:
        return OPEN_SOURCE_MODELS[0]  # Default to Llama 4 Scout
    
    return max(candidates, key=score)


# Test model selection
print("Model Selection Examples:")
print("-" * 50)

tasks = [
    ("code generation", 8000, False, False),
    ("translation", 4000, True, True),
    ("long document analysis", 500_000, False, False),
    ("math reasoning", 4000, False, False),
]

for task, context, multilingual, open_license in tasks:
    model = select_model_for_task(task, context, multilingual, open_license)
    print(f"\nTask: {task}")
    print(f"  Context needed: {context:,} tokens")
    print(f"  Recommended: {model.name}")

## Open-Source Embedding Models

For RAG applications, embedding model selection is critical. The MTEB (Massive Text Embedding Benchmark) leaderboard tracks performance across tasks like retrieval, classification, and clustering.

In [ ]:
class EmbeddingModelInfo(BaseModel):
    """Information about an embedding model."""
    name: str
    provider: str
    dimensions: int
    max_tokens: int
    mteb_avg_score: float
    best_for: list[str]
    license: str


EMBEDDING_MODELS = [
    EmbeddingModelInfo(
        name="Qwen3-Embedding-8B",
        provider="Alibaba",
        dimensions=1024,
        max_tokens=8192,
        mteb_avg_score=72.3,
        best_for=["Multilingual retrieval", "Long documents", "High accuracy"],
        license="Apache 2.0"
    ),
    EmbeddingModelInfo(
        name="BGE-M3",
        provider="BAAI",
        dimensions=1024,
        max_tokens=8192,
        mteb_avg_score=71.8,
        best_for=["Multi-vector retrieval", "Dense + Sparse hybrid", "100+ languages"],
        license="MIT"
    ),
    EmbeddingModelInfo(
        name="text-embedding-3-large",
        provider="OpenAI",
        dimensions=3072,
        max_tokens=8191,
        mteb_avg_score=70.5,
        best_for=["OpenAI ecosystem", "High quality", "Easy integration"],
        license="Proprietary"
    ),
    EmbeddingModelInfo(
        name="nomic-embed-text-v2-moe",
        provider="Nomic AI",
        dimensions=768,
        max_tokens=8192,
        mteb_avg_score=69.8,
        best_for=["Cost-effective", "On-device", "Open weights"],
        license="Apache 2.0"
    ),
]


print("Embedding Model Comparison (MTEB Leaderboard - February 2026)")
print("=" * 70)
print(f"{'Model':<30} {'MTEB':>6} {'Dims':>6} {'Max Tokens':>10}")
print("-" * 70)
for model in sorted(EMBEDDING_MODELS, key=lambda x: -x.mteb_avg_score):
    print(f"{model.name:<30} {model.mteb_avg_score:>6.1f} {model.dimensions:>6} {model.max_tokens:>10,}")

print("\nKey insight: Task-specific performance matters more than overall MTEB score.")
print("Check the leaderboard at: https://huggingface.co/spaces/mteb/leaderboard")

## Deploying Open Models with Together AI

Together AI provides managed infrastructure for open-source models with an OpenAI-compatible API. This gives you the cost benefits of open models without the operational overhead of self-hosting.

In [ ]:
class TogetherAIClient:
    """Wrapper for Together AI API."""
    
    def __init__(self, api_key: str = None):
        self.api_key = api_key or os.getenv("TOGETHER_API_KEY")
        if not self.api_key:
            raise ValueError("TOGETHER_API_KEY required")
        self.client = Together(api_key=self.api_key)
    
    def chat(self, model: str, messages: list[dict], **kwargs) -> str:
        """Send a chat completion request."""
        response = self.client.chat.completions.create(
            model=model,
            messages=messages,
            **kwargs
        )
        return response.choices[0].message.content
    
    def embed(self, model: str, texts: list[str]) -> list[list[float]]:
        """Generate embeddings for texts."""
        response = self.client.embeddings.create(
            model=model,
            input=texts
        )
        return [item.embedding for item in response.data]


# Test Together AI if available
if TOGETHER_API_KEY:
    print("Testing Together AI...")
    together_client = TogetherAIClient()
    
    # Test chat completion
    response = together_client.chat(
        model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
        messages=[{"role": "user", "content": "What is 2+2? Answer with just the number."}],
        max_tokens=10
    )
    print(f"Chat response: {response}")
else:
    print("Together AI not configured.")
    print("To use Together AI:")
    print("  1. Sign up at https://together.ai")
    print("  2. Get your API key")
    print("  3. Set TOGETHER_API_KEY in your .env file")

### Model Fallback Patterns

A robust production system should gracefully fall back when the primary model fails. This pattern uses an open-source model as the primary (lower cost) with a proprietary model as fallback (higher reliability).

In [ ]:
class ModelFallbackChain:
    """Implement fallback from open model to proprietary API."""
    
    def __init__(self, primary_model: str, fallback_model: str):
        self.primary_model = primary_model
        self.fallback_model = fallback_model
        self.primary_client = TogetherAIClient() if TOGETHER_API_KEY else None
        self.fallback_llm = ChatOpenAI(model=fallback_model)
        self.stats = {"primary_calls": 0, "fallback_calls": 0}
    
    def invoke(self, prompt: str) -> tuple[str, str]:
        """Call primary model, fall back on failure.
        
        Returns:
            tuple: (response, model_used)
        """
        # Try primary (Together AI with open-source model)
        if self.primary_client:
            try:
                response = self.primary_client.chat(
                    model=self.primary_model,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=500
                )
                self.stats["primary_calls"] += 1
                return response, self.primary_model
            except Exception as e:
                print(f"Primary model failed: {e}. Falling back...")
        
        # Fallback to OpenAI
        response = self.fallback_llm.invoke(prompt)
        self.stats["fallback_calls"] += 1
        return response.content, self.fallback_model
    
    def get_stats(self) -> dict:
        """Get usage statistics."""
        total = self.stats["primary_calls"] + self.stats["fallback_calls"]
        return {
            **self.stats,
            "total_calls": total,
            "primary_rate": self.stats["primary_calls"] / total if total > 0 else 0
        }


# Demonstration
fallback_chain = ModelFallbackChain(
    primary_model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
    fallback_model="gpt-4o-mini"
)

response, model_used = fallback_chain.invoke("Explain RAG in one sentence.")
print(f"Model used: {model_used}")
print(f"Response: {response[:200]}..." if len(response) > 200 else f"Response: {response}")
print(f"\nStats: {fallback_chain.get_stats()}")

## Performance Benchmarking

When comparing models and providers, measure latency, throughput, and cost systematically.

In [ ]:
class BenchmarkResult(BaseModel):
    """Result of a model benchmark."""
    model: str
    provider: str
    latency_ms: float
    response_length: int
    tokens_per_second: float
    success: bool = True
    error: Optional[str] = None


def benchmark_model(
    name: str,
    provider: str,
    invoke_fn,
    prompt: str
) -> BenchmarkResult:
    """Benchmark a single model invocation."""
    try:
        start = time.perf_counter()
        response = invoke_fn(prompt)
        elapsed = time.perf_counter() - start
        
        # Estimate tokens (rough: ~4 chars per token)
        response_text = response if isinstance(response, str) else response.content
        tokens = len(response_text) // 4
        
        return BenchmarkResult(
            model=name,
            provider=provider,
            latency_ms=elapsed * 1000,
            response_length=len(response_text),
            tokens_per_second=tokens / elapsed if elapsed > 0 else 0,
        )
    except Exception as e:
        return BenchmarkResult(
            model=name,
            provider=provider,
            latency_ms=0,
            response_length=0,
            tokens_per_second=0,
            success=False,
            error=str(e)
        )


# Run benchmarks
BENCHMARK_PROMPT = "Explain the concept of PagedAttention in LLM inference in 3 sentences."

results = []

# OpenAI benchmark
print("Benchmarking models...")
result = benchmark_model(
    "gpt-4o-mini",
    "OpenAI",
    lambda p: llm.invoke(p),
    BENCHMARK_PROMPT
)
results.append(result)
print(f"  OpenAI: {result.latency_ms:.0f}ms")

# Together AI benchmark (if available)
if TOGETHER_API_KEY:
    together_client = TogetherAIClient()
    result = benchmark_model(
        "Llama-4-Scout-17B",
        "Together AI",
        lambda p: together_client.chat(
            model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
            messages=[{"role": "user", "content": p}],
            max_tokens=200
        ),
        BENCHMARK_PROMPT
    )
    results.append(result)
    print(f"  Together AI: {result.latency_ms:.0f}ms")

# Ollama benchmark (if available)
if OLLAMA_RUNNING:
    ollama_llm = ChatOllama(model="llama3.2", temperature=0)
    result = benchmark_model(
        "llama3.2",
        "Ollama (local)",
        lambda p: ollama_llm.invoke(p),
        BENCHMARK_PROMPT
    )
    results.append(result)
    print(f"  Ollama: {result.latency_ms:.0f}ms")

# Display results
print("\nBenchmark Results:")
print("-" * 70)
print(f"{'Provider':<20} {'Model':<25} {'Latency':>10} {'Tokens/s':>10}")
print("-" * 70)
for r in results:
    if r.success:
        print(f"{r.provider:<20} {r.model:<25} {r.latency_ms:>8.0f}ms {r.tokens_per_second:>10.1f}")
    else:
        print(f"{r.provider:<20} {r.model:<25} FAILED: {r.error[:30]}")

## Agent-to-Agent Communication Patterns

When building multi-agent systems at scale, you need robust communication patterns. This section covers three fundamental patterns:

1. **Request-Response** — Synchronous call and wait
2. **Pub-Sub** — Event-driven, decoupled communication
3. **Event Sourcing** — Append-only log for audit trails

In [ ]:
class AgentMessage(BaseModel):
    """Standard message format for agent communication."""
    sender: str
    receiver: str
    message_type: str  # "request", "response", "event"
    payload: dict
    correlation_id: str
    timestamp: str = ""
    
    def __init__(self, **data):
        if "timestamp" not in data or not data["timestamp"]:
            data["timestamp"] = datetime.now().isoformat()
        super().__init__(**data)


# Example message
msg = AgentMessage(
    sender="coordinator",
    receiver="researcher",
    message_type="request",
    payload={"task": "Find information about RAG", "max_results": 5},
    correlation_id=str(uuid.uuid4())
)
print("Example Agent Message:")
print(msg.model_dump_json(indent=2))

### Request-Response Pattern

The simplest pattern: one agent sends a request, another processes it and sends back a response.

In [ ]:
class RequestResponseAgent:
    """Agent that handles request-response communication."""
    
    def __init__(self, name: str, handler=None):
        self.name = name
        self.handler = handler or (lambda payload: {"result": "processed"})
        self.pending_requests: dict[str, AgentMessage] = {}
    
    def send_request(self, receiver: str, payload: dict) -> str:
        """Send a request and return correlation ID."""
        correlation_id = str(uuid.uuid4())
        message = AgentMessage(
            sender=self.name,
            receiver=receiver,
            message_type="request",
            payload=payload,
            correlation_id=correlation_id
        )
        self.pending_requests[correlation_id] = message
        print(f"[{self.name}] Sent request {correlation_id[:8]}... to {receiver}")
        return correlation_id
    
    def handle_request(self, message: AgentMessage) -> AgentMessage:
        """Process an incoming request and return response."""
        print(f"[{self.name}] Processing request from {message.sender}")
        result = self.handler(message.payload)
        return AgentMessage(
            sender=self.name,
            receiver=message.sender,
            message_type="response",
            payload=result,
            correlation_id=message.correlation_id
        )
    
    def receive_response(self, message: AgentMessage):
        """Process an incoming response."""
        if message.correlation_id in self.pending_requests:
            del self.pending_requests[message.correlation_id]
            print(f"[{self.name}] Received response: {message.payload}")
            return message.payload
        return None


# Demonstration
def research_handler(payload):
    """Simple research handler."""
    return {"findings": f"Found 3 sources about {payload.get('topic', 'unknown')}", "confidence": 0.85}


coordinator = RequestResponseAgent("coordinator")
researcher = RequestResponseAgent("researcher", handler=research_handler)

# Simulate communication
print("Request-Response Pattern Demo:")
print("-" * 40)
request_id = coordinator.send_request("researcher", {"topic": "RAG systems"})

# In a real system, this would go through a message queue
request_msg = coordinator.pending_requests[request_id]
response_msg = researcher.handle_request(request_msg)
coordinator.receive_response(response_msg)

### Pub-Sub for Event-Driven Coordination

Publish-subscribe decouples agents: publishers emit events without knowing who listens, subscribers react to events they care about.

In [ ]:
class PubSubCoordinator:
    """Event-driven coordination via publish-subscribe."""
    
    def __init__(self):
        self.subscribers: dict[str, list] = {}  # topic -> [callbacks]
        self.event_log: list[AgentMessage] = []
    
    def subscribe(self, topic: str, callback):
        """Subscribe to events on a topic."""
        if topic not in self.subscribers:
            self.subscribers[topic] = []
        self.subscribers[topic].append(callback)
        print(f"Subscribed to '{topic}'")
    
    def publish(self, topic: str, sender: str, payload: dict):
        """Publish an event to all subscribers."""
        message = AgentMessage(
            sender=sender,
            receiver="broadcast",
            message_type="event",
            payload={"topic": topic, **payload},
            correlation_id=str(uuid.uuid4())
        )
        self.event_log.append(message)
        
        subscribers = self.subscribers.get(topic, [])
        print(f"[{sender}] Published to '{topic}' ({len(subscribers)} subscribers)")
        
        for callback in subscribers:
            callback(message)
    
    def get_event_count(self) -> int:
        """Return total events published."""
        return len(self.event_log)


# Demonstration
print("Pub-Sub Pattern Demo:")
print("-" * 40)

coordinator = PubSubCoordinator()

# Subscribe handlers
def on_task_complete(msg):
    print(f"  [Logger] Task completed: {msg.payload.get('task_id')}")

def on_task_complete_notify(msg):
    print(f"  [Notifier] Sending notification for task {msg.payload.get('task_id')}")

coordinator.subscribe("task.complete", on_task_complete)
coordinator.subscribe("task.complete", on_task_complete_notify)

# Publish events
coordinator.publish("task.complete", "worker-1", {"task_id": "T-001", "result": "success"})
coordinator.publish("task.complete", "worker-2", {"task_id": "T-002", "result": "success"})

print(f"\nTotal events: {coordinator.get_event_count()}")

### Event Sourcing for Audit Trails

Event sourcing stores every state change as an immutable event. This provides a complete audit trail and enables replay for debugging or recovery.

In [ ]:
class EventStore:
    """Append-only event store for audit trails."""
    
    def __init__(self):
        self.events: list[dict] = []
    
    def append(self, event_type: str, agent: str, data: dict) -> dict:
        """Append an event to the store."""
        event = {
            "id": len(self.events),
            "timestamp": datetime.now().isoformat(),
            "type": event_type,
            "agent": agent,
            "data": data
        }
        self.events.append(event)
        return event
    
    def get_events_for_agent(self, agent: str) -> list[dict]:
        """Retrieve all events for a specific agent."""
        return [e for e in self.events if e["agent"] == agent]
    
    def get_events_by_type(self, event_type: str) -> list[dict]:
        """Retrieve all events of a specific type."""
        return [e for e in self.events if e["type"] == event_type]
    
    def replay(self, from_id: int = 0) -> list[dict]:
        """Replay events from a specific point."""
        return self.events[from_id:]


# Demonstration
print("Event Sourcing Demo:")
print("-" * 40)

store = EventStore()

# Simulate a workflow
store.append("task.created", "coordinator", {"task": "Research LLM servers", "priority": "high"})
store.append("task.assigned", "coordinator", {"task_id": 0, "assignee": "researcher"})
store.append("task.started", "researcher", {"task_id": 0})
store.append("task.progress", "researcher", {"task_id": 0, "progress": 50})
store.append("task.completed", "researcher", {"task_id": 0, "result": "Found 5 relevant papers"})

print(f"Total events: {len(store.events)}")
print(f"\nEvents by researcher:")
for event in store.get_events_for_agent("researcher"):
    print(f"  [{event['type']}] {event['data']}")

print(f"\nReplay from event 2:")
for event in store.replay(2):
    print(f"  {event['id']}: [{event['type']}] {event['agent']}")

## Exposing Agents via MCP

The Model Context Protocol (MCP) provides a standard way to expose your agent's capabilities to other agents and applications. An MCP server exposes:
- **Tools** — Callable functions (like POST requests)
- **Resources** — Read-only data (like GET requests)
- **Prompts** — Reusable templates

We'll build a simple Task Tracker MCP server to demonstrate these concepts.

In [ ]:
# Create the mcp_servers directory
os.makedirs("mcp_servers", exist_ok=True)
print("Created mcp_servers/ directory")

In [ ]:
%%writefile mcp_servers/task_tracker_server.py
"""Task Tracker MCP Server - Simple task management for agents."""
import json
import uuid
from datetime import datetime
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("TaskTracker")

# In-memory task storage
TASKS = {}


@mcp.tool()
def create_task(title: str, description: str = "", priority: str = "medium") -> str:
    """Create a new task with title, description, and priority (low/medium/high)."""
    task_id = f"TASK-{len(TASKS) + 1:03d}"
    TASKS[task_id] = {
        "id": task_id,
        "title": title,
        "description": description,
        "priority": priority,
        "status": "pending",
        "created_at": datetime.now().isoformat(),
        "completed_at": None
    }
    return json.dumps({"created": TASKS[task_id]})


@mcp.tool()
def list_tasks(status: str = "all") -> str:
    """List tasks filtered by status (all/pending/completed)."""
    if status == "all":
        tasks = list(TASKS.values())
    else:
        tasks = [t for t in TASKS.values() if t["status"] == status]
    return json.dumps({"tasks": tasks, "count": len(tasks)})


@mcp.tool()
def complete_task(task_id: str) -> str:
    """Mark a task as completed by its ID."""
    if task_id not in TASKS:
        return json.dumps({"error": f"Task {task_id} not found"})
    TASKS[task_id]["status"] = "completed"
    TASKS[task_id]["completed_at"] = datetime.now().isoformat()
    return json.dumps({"completed": TASKS[task_id]})


@mcp.tool()
def get_task(task_id: str) -> str:
    """Get details for a specific task."""
    if task_id not in TASKS:
        return json.dumps({"error": f"Task {task_id} not found"})
    return json.dumps(TASKS[task_id])


@mcp.resource("tasks://summary")
def get_summary() -> str:
    """Get summary of all tasks (counts by status)."""
    pending = sum(1 for t in TASKS.values() if t["status"] == "pending")
    completed = sum(1 for t in TASKS.values() if t["status"] == "completed")
    return json.dumps({
        "total": len(TASKS),
        "pending": pending,
        "completed": completed
    })


if __name__ == "__main__":
    mcp.run()

### Testing the MCP Server

We connect to our Task Tracker server using the MCP SDK and exercise its tools.

In [ ]:
# Test the Task Tracker MCP server
server_params = StdioServerParameters(
    command=sys.executable,
    args=["mcp_servers/task_tracker_server.py"]
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # Discover tools
        tools = await session.list_tools()
        print("Task Tracker MCP Tools:")
        for tool in tools.tools:
            print(f"  - {tool.name}: {tool.description[:60]}...")
        
        # Create some tasks
        print("\n--- Creating tasks ---")
        result = await session.call_tool("create_task", {
            "title": "Review PR #123",
            "description": "Review the MCP integration PR",
            "priority": "high"
        })
        print(f"Created: {result.content[0].text}")
        
        result = await session.call_tool("create_task", {
            "title": "Update documentation",
            "priority": "medium"
        })
        print(f"Created: {result.content[0].text}")
        
        # List all tasks
        print("\n--- All tasks ---")
        result = await session.call_tool("list_tasks", {"status": "all"})
        print(result.content[0].text)
        
        # Complete a task
        print("\n--- Completing task ---")
        result = await session.call_tool("complete_task", {"task_id": "TASK-001"})
        print(result.content[0].text)
        
        # Get summary
        print("\n--- Summary ---")
        result = await session.read_resource("tasks://summary")
        print(result.contents[0].text)

print("\nMCP server test complete!")

### A2A Protocol and Agent Cards

The Agent-to-Agent (A2A) protocol enables agents to discover each other's capabilities. An Agent Card describes what an agent can do.

In [ ]:
class AgentCard(BaseModel):
    """Agent Card for capability discovery (A2A Protocol)."""
    name: str
    description: str
    version: str
    capabilities: list[str]
    supported_modalities: list[str]  # text, audio, video, image
    endpoint: str
    authentication: dict
    metadata: dict = {}


# Task Tracker Agent Card
task_tracker_card = AgentCard(
    name="TaskTracker",
    description="Simple task management agent for tracking and organizing work",
    version="1.0.0",
    capabilities=[
        "create_task",
        "list_tasks",
        "complete_task",
        "get_task"
    ],
    supported_modalities=["text"],
    endpoint="mcp://localhost:8000/task-tracker",
    authentication={"type": "none"},
    metadata={
        "author": "AI Engineering Bootcamp",
        "tags": ["productivity", "tasks", "organization"]
    }
)

print("Agent Card:")
print(task_tracker_card.model_dump_json(indent=2))

In [ ]:
class A2AClient:
    """Client for Agent-to-Agent communication via A2A protocol."""
    
    def __init__(self, agent_card: AgentCard):
        self.agent_card = agent_card
    
    def discover_capabilities(self) -> list[str]:
        """Discover what the remote agent can do."""
        return self.agent_card.capabilities
    
    def can_handle(self, capability: str) -> bool:
        """Check if agent supports a capability."""
        return capability in self.agent_card.capabilities
    
    def get_endpoint(self) -> str:
        """Get the agent's endpoint."""
        return self.agent_card.endpoint


# Demonstration: Agent discovery
print("A2A Discovery Demo:")
print("-" * 40)

client = A2AClient(task_tracker_card)

print(f"Agent: {task_tracker_card.name}")
print(f"Endpoint: {client.get_endpoint()}")
print(f"Capabilities: {client.discover_capabilities()}")
print(f"\nCan handle 'create_task'? {client.can_handle('create_task')}")
print(f"Can handle 'delete_task'? {client.can_handle('delete_task')}")

## Bringing It All Together

Let's combine all the concepts from this chapter into a complete demonstration.

In [ ]:
print("=" * 60)
print("ADVANCED INFRASTRUCTURE: COMPLETE DEMO")
print("=" * 60)

# 1. Model Selection
print("\n1. MODEL SELECTION")
print("-" * 40)
task = "Generate a summary of LLM server options"

if OLLAMA_RUNNING:
    model_choice = "Ollama (local)"
    provider = "local"
elif TOGETHER_API_KEY:
    model_choice = "Llama-4-Scout via Together AI"
    provider = "together"
else:
    model_choice = "gpt-4o-mini via OpenAI"
    provider = "openai"

print(f"Task: {task}")
print(f"Selected: {model_choice}")

# 2. Execute with Fallback
print("\n2. EXECUTION WITH FALLBACK")
print("-" * 40)
fallback = ModelFallbackChain(
    primary_model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
    fallback_model="gpt-4o-mini"
)
response, model_used = fallback.invoke("List 3 popular LLM servers in one sentence.")
print(f"Model used: {model_used}")
print(f"Response: {response}")

# 3. MCP Server Exposure
print("\n3. MCP SERVER EXPOSURE")
print("-" * 40)

server_params = StdioServerParameters(
    command=sys.executable,
    args=["mcp_servers/task_tracker_server.py"]
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await session.list_tools()
        print(f"Exposed {len(tools.tools)} tools via MCP:")
        for tool in tools.tools:
            print(f"  - {tool.name}")

# 4. Agent Communication
print("\n4. AGENT-TO-AGENT COORDINATION")
print("-" * 40)

event_store = EventStore()
event_store.append("demo.started", "coordinator", {"chapter": 13})
event_store.append("model.selected", "coordinator", {"model": model_used})
event_store.append("mcp.exposed", "task-tracker", {"tools_count": len(tools.tools)})
event_store.append("demo.completed", "coordinator", {"success": True})

print(f"Event log has {len(event_store.events)} events:")
for event in event_store.events:
    print(f"  [{event['type']}] {event['agent']}: {event['data']}")

# 5. Cost Summary
print("\n5. COST COMPARISON")
print("-" * 40)
analyze_costs(10_000_000, 5_000_000)

print("\n" + "=" * 60)
print("DEMO COMPLETE")
print("=" * 60)